In [10]:
from gurobipy import Model, multidict, GRB

# Define the list of DMUs
DMU = ["A", "B", "C", "D", "E"]

# Create dictionaries to store various information
E = {}
val_p1, val_p2, val_p3, val_s1, val_s2 = {}, {}, {}, {}, {}
slack_p1, slack_p2, slack_p3 = {}, {}, {}

In [11]:
# Define the number of inputs (I) and outputs (O) for each DMU
I = 2
O = 3

# Define data specific to each DMU
DMU, Totx1, Totx2 = multidict(
    {
        "A": [11, 14],
        "B": [7, 7],
        "C": [11, 14],
        "D": [14, 14],
        "E": [14, 15],
    }
)

# Define more data specific to each DMU (process 1)
DMU, proc1x1, proc1x2, proc1TotyO, proc1yO, proc1yI = multidict(
    {
        "A": [3, 5, 4, 2, 2],
        "B": [2, 3, 2, 1, 1],
        "C": [3, 4, 2, 1, 1],
        "D": [4, 6, 3, 2, 1],
        "E": [5, 6, 4, 3, 1],
    }
)

# Define more data specific to each DMU (process 2)
DMU, proc2x1, proc2x2, proc2TotyO, proc2yO, proc2yI = multidict(
    {
        "A": [4, 3, 3, 2, 1],
        "B": [2, 1, 2, 1, 1],
        "C": [5, 3, 2, 1, 1],
        "D": [5, 5, 4, 3, 1],
        "E": [5, 4, 4, 2, 2],
    }
)

# Define more data specific to each DMU (process 3)
DMU, proc3x1, proc3x2, proc3TotyO = multidict(
    {
        "A": [4, 6, 1],
        "B": [3, 3, 1],
        "C": [3, 7, 2],
        "D": [5, 3, 1],
        "E": [4, 5, 3],
    }
)

P1, P2, P3 = {}, {}, {}
v, u = {}, {}





In [12]:
for k in DMU:
    # Create the Gurobi model
    m = Model("network_DEA")

    # Define decision variables
    v = {i: m.addVar(vtype=GRB.CONTINUOUS, name=f"v_{i}") for i in range(I)}
    u = {i: m.addVar(vtype=GRB.CONTINUOUS, name=f"u_{i}") for i in range(O)}
    
    # Update the model
    m.update()

    # Set the objective function to maximize efficiency
    m.setObjective(
        u[0] * proc1yO[k] + u[1] * proc2yO[k] + u[2] * proc3TotyO[k], GRB.MAXIMIZE
    )

    # Add constraints
    m.addConstr(v[0] * Totx1[k] + v[1] * Totx2[k] == 1)

    for j in DMU:
        P1[j] = m.addConstr(
            u[0] * proc1TotyO[j] - (v[0] * proc1x1[j] + v[1] * proc1x2[j]) <= 0
        )
        P2[j] = m.addConstr(
            u[1] * proc2TotyO[j] - (v[0] * proc2x1[j] + v[1] * proc2x2[j]) <= 0
        )
        P3[j] = m.addConstr(
            u[2] * proc3TotyO[j]
            - (
                v[0] * proc3x1[j]
                + v[1] * proc3x2[j]
                + u[0] * proc1yI[j]
                + u[1] * proc2yI[j]
            )
            <= 0
        )

    # Optimize the model
    m.optimize()

    # Store efficiency values in the dictionary
    E[k] = f"The efficiency of DMU {k}: {m.objVal}"

    u_sol = m.getAttr("x", u)
    v_sol = m.getAttr("x", v)

    # Calculate process and stage efficiencies
    E1 = u_sol[0] * proc1TotyO[k] / (v_sol[0] * proc1x1[k] + v_sol[1] * proc1x2[k])
    E2 = u_sol[1] * proc2TotyO[k] / (v_sol[0] * proc2x1[k] + v_sol[1] * proc2x2[k])
    E3 = (
        u_sol[2]
        * proc3TotyO[k]
        / (
            v_sol[0] * proc3x1[k]
            + v_sol[1] * proc3x2[k]
            + u_sol[0] * proc1yI[k]
            + u_sol[1] * proc2yI[k]
        )
    )
    stage1 = (
        u_sol[0] * proc1TotyO[k]
        + u_sol[1] * proc2TotyO[k]
        + v_sol[0] * proc3x1[k]
        + v_sol[1] * proc3x2[k]
    ) / (v_sol[0] * Totx1[k] + v_sol[1] * Totx2[k])
    stage2 = (
        u_sol[0] * proc1yO[k] + u_sol[1] * proc2yO[k] + u_sol[2] * proc3TotyO[k]
    ) / (
        u_sol[0] * proc1TotyO[k]
        + u_sol[1] * proc2TotyO[k]
        + v_sol[0] * proc3x1[k]
        + v_sol[1] * proc3x2[k]
    )

    # Store process and stage efficiency values
    val_p1[k] = f"The efficiency of process 1 of DMU {k}: {E1:.4g}"
    val_p2[k] = f"The efficiency of process 2 of DMU {k}: {E2:.4g}"
    val_p3[k] = f"The efficiency of process 3 of DMU {k}: {E3:.4g}"
    val_s1[k] = f"The efficiency of stage 1 of DMU {k}: {stage1:.4g}"
    val_s2[k] = f"The efficiency of stage 2 of DMU {k}: {stage2:.4g}"

    # Calculate and store inefficiency values
    process1_slack = m.getAttr("slack", P1)
    slack_p1[k] = f"The inefficiency of process 1 of DMU {k}: {process1_slack[k]:.4g}"
    process2_slack = m.getAttr("slack", P2)
    slack_p2[k] = f"The inefficiency of process 2 of DMU {k}: {process2_slack[k]:.4g}"
    process3_slack = m.getAttr("slack", P3)
    slack_p3[k] = f"The inefficiency of process 3 of DMU {k}: {process3_slack[k]:.4g}"

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 5 columns and 57 nonzeros
Model fingerprint: 0xaff3bd3b
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 9 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 4 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.5792210e-01   4.010462e-02   0.000000e+00      0s
       3    5.2272727e-01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.227272727e-01
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 5 columns and 

In [13]:
# Display the results for each DMU
for k in DMU:
    print(E[k])
    print(val_p1[k])
    print(val_p2[k])
    print(val_p3[k])
    print(val_s1[k])
    print(val_s2[k])
    print(slack_p1[k])
    print(slack_p2[k])
    print(slack_p3[k])
    print("--------------")

The efficiency of DMU A: 0.5227272727272727
The efficiency of process 1 of DMU A: 1
The efficiency of process 2 of DMU A: 0.75
The efficiency of process 3 of DMU A: 0.3462
The efficiency of stage 1 of DMU A: 0.9091
The efficiency of stage 2 of DMU A: 0.575
The inefficiency of process 1 of DMU A: 0
The inefficiency of process 2 of DMU A: 0.09091
The inefficiency of process 3 of DMU A: 0.3864
--------------
The efficiency of DMU B: 0.5952380952380953
The efficiency of process 1 of DMU B: 0.8333
The efficiency of process 2 of DMU B: 1
The efficiency of process 3 of DMU B: 0.5088
The efficiency of stage 1 of DMU B: 0.9286
The efficiency of stage 2 of DMU B: 0.641
The inefficiency of process 1 of DMU B: 0.07143
The inefficiency of process 2 of DMU B: 0
The inefficiency of process 3 of DMU B: 0.3333
--------------
The efficiency of DMU C: 0.5681818181818181
The efficiency of process 1 of DMU C: 0.5
The efficiency of process 2 of DMU C: 0.4
The efficiency of process 3 of DMU C: 0.9474
The eff